# First explorations with one Qubit

In [2]:
from qiskit import QuantumCircuit, transpile
from qiskit_aer import Aer
from qiskit.quantum_info import Statevector
from math import sqrt

## Hello Quantum World (Preparing a one Qubit System with an initial state)

In Quantum Commputing, the minimal unit of information is the Qubit, which formally is a normalized vector in $\mathbb{C}^2$.  We make the definitions $\ket{0} := [1, 0]^T$, $\ket{1} := [0, 1]^T$. In Quantum Computing, the set $\{\ket{0}, \ket{1} \}$ is known as the computational basis.  
We can represent a Qubit in $\mathbb{C}^2$ as a linear combination of the vectors in the computational basis, which then has the general form $\alpha_0\ket{0} + \alpha_1\ket{1}$. It is aslo asked that the state of a Qubit is normed; that is, $\lvert \alpha_0\rvert^2 + \lvert \alpha_1\rvert^2 = 1$.  
We first instatiate a quantum circuit with only one qubit:

In [4]:
circuit = QuantumCircuit(1) # The parameter 1 indicates we are dealing with a one-qubit system

Now, let us prepare the Qubit with the state vectors from the computaional basis of $\mathbb{C}^2$ ($\ket{0}$ and $\ket{1}$) and access the state vector of this Qubit:

In [6]:
for base_vector in ['0', '1']:
    state_vector = Statevector.from_label(base_vector)
    state_vector = state_vector.evolve(circuit)
    print(f'Qubit initialized with the state:', state_vector)

Qubit initialized with the state: Statevector([1.+0.j, 0.+0.j],
            dims=(2,))
Qubit initialized with the state: Statevector([0.+0.j, 1.+0.j],
            dims=(2,))


In Quantum Computing, the basis $\{\ket{+}, \ket{-} \}$, where $\ket{+} := \dfrac{1}{\sqrt{2}}(\ket{0} + \ket{1})$ and $\ket{-} := \dfrac{1}{\sqrt{2}}(\ket{0} - \ket{1})$ is kown as the Hadamard basis of $\mathbb{C}^2$.  
We now initialiaze the Qubit with the state vectors from the Hadamard basis and access the state vector of this Qubit:

In [8]:
for base_vector in ['+', '-']:
    state_vector = Statevector.from_label(base_vector)
    state_vector = state_vector.evolve(circuit)
    print(f'Qubit initialized with the state:', state_vector)

Qubit initialized with the state: Statevector([0.70710678+0.j, 0.70710678+0.j],
            dims=(2,))
Qubit initialized with the state: Statevector([ 0.70710678+0.j, -0.70710678+0.j],
            dims=(2,))


## Measuring a One-Qubit System

In the previous examples, we cheated because we accessed the state of the quantum system which we had previously prepared. In practice, after preparing a quantum system, we cannot access the state of the system, but we have to **measure** the system with respect to an **obervable** $A: \mathcal{H} \rightarrow \mathcal{H}$, which is represented mathematically as a self-adjoint linear operator on $\mathcal{H}$, the Hilbert Space used to represent the states of the quantum system.  
The spectral theorem in linear algebra tells us that there is an orthonormal basis of $\mathcal{H}$ formed by the eigenvectors of $A$. For a one-qubit quantum system, $\mathcal{H} = \mathbb{C}^2$ and we can represent any state $\psi$ of the qubit as 
$$\psi = \alpha_0\ket{v_0} + \alpha_1\ket{v_1},$$
where $v_0$, $v_1$ are orthonormal eigenvectors of $A$ and $\lvert \alpha_0\rvert^2 + \lvert \alpha_1\rvert^2 = 1$.  
Now, the postulates of Quantum Mechanics tells us that if $\lambda_0$, $\lambda_1$ are the eigenvlaues of $A$ corresponding to the eigenvectors $v_0$, $v_1$, then the result of the measurement will be either $\lambda_0$ or $\lambda_1$ wih respective probabilities $\lvert\alpha_0\rvert^2$, $\lvert\alpha_1\rvert^2$. If the output of the masure is $\alpha_i$, that means the system has "collpased" to the state $\ket{v_i}$ as a result of the measurement.  

In order to explore this theory with the help of Qiskit, let us now instantiate a Quantum Circuit with one qubit and a classical bit to store the results of our measurements.

In [11]:
circuit = QuantumCircuit(1, 1)

We now prepare the Qubit with the state $\ket{0} = [1, 0]^T$. 

In [13]:
circuit.initialize([1,0], 0, 1)

- The list given as first parameter indicates the desired coefficients of the state to prepare with respect to the computational basis $\{\ket{0}, \ket{1} \}$.
- The second parameter indicates which Qubit will attain this state (in this example $0$ points to the first and only Qubit in the quantum circuit)
- The third parameter indicates if the coordinates given in the first parameter are to be normed ($1=True$, $0=False$), so that the resulting state has norm 1 .

We will now simulate the process of measuring this qubit 10000 times and print the results obtained:  

In [16]:
# Measure the qubit and store the result in the classical bit
circuit.measure(0, 0)
print(circuit)
 # For local simulation
simulator = Aer.get_backend('qasm_simulator')
# Transpile and run the circuit
shots = 10000
t_qc = transpile(circuit, simulator)
job = simulator.run(t_qc, shots=shots)
result = job.result()
counts = result.get_counts()
print(counts)

     ┌─────────────────┐┌─┐
  q: ┤ Initialize(1,0) ├┤M├
     └─────────────────┘└╥┘
c: 1/════════════════════╩═
                         0 
{'0': 10000}


[In Qiskit, measurements are performed in the computational basis](https://quantum.cloud.ibm.com/docs/de/guides/measure-qubits). In other words, measurements are performed with respect to the Pauli Matrix 
$
\sigma_z :=
\begin{bmatrix}
    1 &  0 \\
    0 & -1 \\
\end{bmatrix}
$

Now, it is easy to verify that $\sigma_z$ has $\ket{0}$, $\ket{1}$ as eigenvectors with corresponding eigenvalues $1$, $-1$. The output of the previous cell tells us that after preparing the a qubit with the state $\ket{0}$ and measuring it $10,000$ times with respect to $\sigma_z$ we obtained the eigenvalue corresponding to $\ket{0}$ all of the times, which is consistent with the postulates of quantum mechanics.

Let's repeat the previous experiment preparing the qubit with the state $\ket{1}$. Of course this time we expect to measure the eigenvalue corresponding to $\ket{1}$ every time. 

In [19]:
circuit = QuantumCircuit(1, 1)
circuit.initialize([0, 1], 0, 1)
# Measure the qubit and store the result in the classical bit
circuit.measure(0, 0)
print(circuit)
 # For local simulation
simulator = Aer.get_backend('qasm_simulator')
# Transpile and run the circuit
shots = 10000
t_qc = transpile(circuit, simulator)
job = simulator.run(t_qc, shots=shots)
result = job.result()
counts = result.get_counts()
print(counts)

     ┌─────────────────┐┌─┐
  q: ┤ Initialize(0,1) ├┤M├
     └─────────────────┘└╥┘
c: 1/════════════════════╩═
                         0 
{'1': 10000}


What happens if we repeat the process, this time preparing the Qubit with the sate $\ket{+} = \dfrac{1}{\sqrt{2}}(\ket{0} + \ket{1})$ (which is a superposition of the state vectors from the computational basis)? This time the theory tells us that we should measure each of the eigenvalues corresponding to $\ket{0}$, $\ket{1}$ more or less half of the time.

In [21]:
circuit = QuantumCircuit(1, 1)
circuit.initialize([1/sqrt(2), 1/sqrt(2)], 0, 1)
# Measure the qubit and store the result in the classical bit
circuit.measure(0, 0)
print(circuit)
 # For local simulation
simulator = Aer.get_backend('qasm_simulator')
# Transpile and run the circuit
shots = 10000
t_qc = transpile(circuit, simulator)
job = simulator.run(t_qc, shots=shots)
result = job.result()
counts = result.get_counts()
print(counts)

     ┌─────────────────────────────┐┌─┐
  q: ┤ Initialize(0.70711,0.70711) ├┤M├
     └─────────────────────────────┘└╥┘
c: 1/════════════════════════════════╩═
                                     0 
{'0': 4983, '1': 5017}
